In [1]:
import numpy as np
import pandas as pd

## Punto 1

In [2]:
# before normalizing the loan_lenders.csv table, let's check that each username
# doesn't contain spaces or commas
data_folder_path = 'additional-kiva-snapshot/'
lenders = pd.read_csv(data_folder_path + 'lenders.csv')
pd.set_option('display.max_rows', 10)
lenders

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
2349169,janet7309,Janet,NaN,NaN,NaN,1342097163,NaN,NaN,NaN,NaN,0
2349170,pj4198,NaN,NaN,NaN,NaN,1342097515,NaN,NaN,NaN,NaN,0
2349171,maria2141,Maria,NaN,NaN,US,1342099723,NaN,NaN,2.0,NaN,0
2349172,simone9846,Simone,NaN,NaN,NaN,1342100213,NaN,NaN,NaN,NaN,0


In [3]:
space = lenders['permanent_name'].str.find(' ') == -1
comma = lenders['permanent_name'].str.find(',') == -1
total = space & comma
del space, comma
assert np.any(total), 'Invalid account name'
del total

In [4]:
loan_lenders = pd.read_csv(data_folder_path + 'loans_lenders.csv')
loan_lenders

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."
...,...,...
1387427,678999,"michael43411218, carol5987, gooddogg1, chris41..."
1387428,1207353,"rjhoward1986, jeffrey6870, trolltech4460, elys..."
1387429,1206220,"vicky7746, gooddogg1, fairspirit, craig9729960..."
1387430,1206425,"rich6705, sergiiy9766, angela7509, barbara5610..."


In [5]:
lenders_list = loan_lenders.lenders.str.split(', ')
lengths = [len(l) for l in lenders_list.values]
#np.hstack(prova[0:10])
lengths
norm_loan_lenders = pd.DataFrame({'loan_id': np.repeat(loan_lenders['loan_id'].values, lengths),
                       'lender': np.concatenate(lenders_list)})
norm_loan_lenders

,loan_id,lender
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
28293926,1206425,trogdorfamily7622
28293927,1206425,danny6470
28293928,1206425,don6118
28293929,1206486,alan5175


# Secondo Punto

In [69]:
loans = pd.read_csv(data_folder_path + 'loans.csv', nrows = 100)
pd.set_option('display.max_columns', 100)
loans

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,to buy additional inventories for her sari-sa...,PH,Philippines,Ozamiz - Ozamiz City,shared,0.1,PHP,126.0,2014-01-15 02:23:45.000 +0000,2014-02-14 03:30:06.000 +0000,2013-12-22 08:00:00.000 +0000,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,"To buy American clothing such as blouses, shir...",HN,Honduras,"La Lopez, Choloma",shared,0.1,HNL,201.0,2014-01-14 20:23:20.000 +0000,2014-03-26 22:25:07.000 +0000,2013-12-20 08:00:00.000 +0000,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,"to buy stock of rice, sugar and flour",PK,Pakistan,Lala Musa,shared,0.1,PKR,245.0,2014-01-16 11:32:58.000 +0000,2014-02-15 21:10:05.000 +0000,2014-01-09 08:00:00.000 +0000,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,"to buy cucumber and tomato seeds, as well as f...",KG,Kyrgyzstan,"Aravan village, Osh region",shared,0.1,KGS,171.0,2014-01-20 09:59:48.000 +0000,2014-02-21 03:10:02.000 +0000,2014-01-17 08:00:00.000 +0000,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,to buy organic fertilizer and agrochemical pr...,PH,Philippines,"Baleleng, Sto. Thomas, Isabela",shared,0.1,PHP,123.0,2014-01-14 05:46:21.000 +0000,2014-02-13 06:10:02.000 +0000,2013-12-17 08:00:00.000 +0000,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,961094,Ma. Minda,English,"Ma. Minda is a 49-year-old woman from Tubigon,...","Ma. Minda is a 49-year-old woman from Tubigon,...",150.0,150.0,funded,Weaving,Arts,to buy buri leaves for weaving.,PH,Philippines,"Tubigon, Bohol",shared,0.1,PHP,125.0,2015-10-11 06:51:24.000 +0000,2015-11-11 20:50:02.000 +0000,2015-09-10 07:00:00.000 +0000,2015-10-13 06:14:12.000 +0000,10.0,5,2,1,"#Fabrics, #Schooling, #Parent, #Woman Owned Biz",female,true,irregular,field_partner
96,961161,Usman,English,Usman is a Babban Gona maize farmer. Thanks to...,Usman is a Babban Gona maize farmer. Thanks to...,200.0,200.0,funded,Personal Expenses,Personal Use,"to buy food and clothing, eliminating pressure...",NG,Nigeria,Kaduna,shared,0.1,NGN,288.0,2015-10-11 17:36:53.000 +0000,2015-11-12 14:30:11.000 +0000,2015-12-07 08:00:00.000 +0000,2015-11-12 00:39:38.000 +0000,14.0,8,2,2,#Vegan,male,true,bullet,field_partner
97,961521,Lydia,English,Lydia is a married parent of three children. S...,Lydia is a married parent of three children. S...,550.0,550.0,funded,Beauty Salon,Services,"to buy wigs, shampoo and cream that she will u...",GH,Ghana,Chorkor,shared,0.1,GHS,231.0,2015-10-12 09:22:24.000 +0000,2015-11-12 21:10:06.000 +0000,2015-10-02 07:00:00.000 +0000,2015-10-18

Le due colonne di interesse sono inizialmente codificate come stringhe di testo, è necessario convertirle in un formato adatto a dati temporali. Eventuali valori nulli o non validi, sono automaticamente gestiti da Pandas e diventano <pandas._libs.tslibs.nattype.NaTType>. Per esempio:

In [70]:
print('Esempio di dato mancante:', loans['disburse_time'][64])
loans['disburse_time'] = pd.to_datetime(loans['disburse_time'])
loans['planned_expiration_time'] = pd.to_datetime(loans['planned_expiration_time'])
print('... convertito in NaT', loans['disburse_time'][64])

Esempio di dato mancante: 2010-11-18 08:00:00.000 +0000
... convertito in NaT 2010-11-18 08:00:00+00:00


I NaT si propagano come i nan per i float, per cui qualsiasi operazione algebrica su di essi restisce NaT. Calcolando la differenza sottostante quindi le entrate NaT rimarranno tali

In [71]:
print('Propagazione dei NaT:', (loans['disburse_time'] - loans['planned_expiration_time'])[64])

Propagazione dei NaT: NaT


In [72]:
loans['duration'] = loans['disburse_time'] - loans['planned_expiration_time']

In [74]:
loans[['planned_expiration_time', 'disburse_time', 'duration']]

planned_expiration_time    datetime64[ns, UTC]
disburse_time              datetime64[ns, UTC]
duration                       timedelta64[ns]
dtype: object

# Terzo Punto

Quello che segue è il metodo più rapido per ottenere quali utenti abbiano presato denaro più di una volta. Alternativamente, si possono contare le occorrenze di ciascun nome nella tabella norm_loan_lenders, per poi tenere solo quelli con conteggio > 1. Tuttavia, se il database è coerente i due metodi dovrebbero identificare gli stessi utenti. Il primo metodo è più rapido da scrivere ed eseguire, e permette di restituire anche altri dati dei soggetti senza ulteriori query.

In [122]:
lenders[lenders['loan_purchase_num'] > 1]

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0
6,rene7585,Rene,NaN,NaN,NaN,1461301636,NaN,NaN,2.0,NaN,0
7,harald2826,Harald,NaN,NaN,NaN,1461301670,NaN,NaN,2.0,NaN,0
11,jennifer4328,Jennifer,NaN,NaN,NaN,1461302712,NaN,NaN,3.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
2349158,rakhi,Rakhi,New York,New York,US,1342100607,Student,I care.,4.0,NaN,0
2349162,james75291930,James,NaN,NaN,NaN,1342096854,NaN,NaN,6.0,NaN,0
2349164,carol8279,Carol,NaN,NaN,NaN,1342099416,NaN,NaN,5.0,NaN,0
2349166,eric91401545,Eric,NaN,NaN,NaN,1342100719,NaN,NaN,2.0,NaN,0


# Quarto Punto

Si può interrogare equivalentemente con 'country_code'. Se la tabella è fatta in modo corretto, i due metodi sono equivalenti.

In [25]:
loans['country_name'].value_counts()

Philippines    28
Kenya           7
Nicaragua       5
El Salvador     5
Honduras        4
               ..
Indonesia       1
Georgia         1
Rwanda          1
Mali            1
Israel          1
Name: country_name, Length: 36, dtype: int64

# Quinto Punto

Il codice seguente permette di soddisfare la richiesta. Si noti che le cifre sono tutte in dollari USD.

In [118]:
loan_amount_by_ctry = loans.groupby(by = 'country_name').aggregate({'funded_amount': 'sum'})
loan_amount_by_ctry

,funded_amount
country_name,
Armenia,1125.0
Azerbaijan,2550.0
Bolivia,1925.0
Burundi,1825.0
Cambodia,1850.0
...,...
Tanzania,925.0
Togo,1825.0
Uganda,1975.0


# Sesto Punto

In [119]:
loan_amount_by_ctry['funded_amount_(%)'] = loan_amount_by_ctry['funded_amount']/sum(loan_amount_by_ctry['funded_amount'])
loan_amount_by_ctry

,funded_amount,funded_amount_(%)
country_name,,
Armenia,1125.0,0.015040
Azerbaijan,2550.0,0.034091
Bolivia,1925.0,0.025735
Burundi,1825.0,0.024398
Cambodia,1850.0,0.024733
...,...,...
Tanzania,925.0,0.012366
Togo,1825.0,0.024398
Uganda,1975.0,0.026404


# Settimo Punto

In [140]:
loans_amount_by_yr_ctry = pd.DataFrame({'loan_count': loans.groupby(['country_name', loans.disburse_time.dt.year]).size()})
loans_amount_by_yr_ctry['funded_amount'] = loans.groupby(['country_name', loans.disburse_time.dt.year]).aggregate({'funded_amount': 'sum'})
loans_amount_by_yr_ctry['funded_amount_(%)'] = loans_amount_by_yr_ctry['funded_amount']/sum(loans_amount_by_yr_ctry['funded_amount'])
loans_amount_by_yr_ctry

loan_count  funded_amount  funded_amount_(%)
country_name disburse_time                                              
Armenia      2015                    1         1125.0           0.015040
Azerbaijan   2012                    1         2550.0           0.034091
Bolivia      2012                    1          325.0           0.004345
             2013                    1          875.0           0.011698
             2016                    1          725.0           0.009693
...                                ...            ...                ...
Togo         2012                    2         1825.0           0.024398
Uganda       2012                    2          875.0           0.011698
             2015                    2         1100.0           0.014706
Vietnam      2016                    1         3750.0           0.050134
Yemen        2014                    1         2350.0           0.031417

[61 rows x 3 columns]